In [1]:
import pandas as pd
import yfinance as yf
import datetime

In [18]:
# La mitad de la inversion en moneda de interes (BTC)
TOKEN = 0.002

In [2]:
symbol="ETH-BTC"
data = pd.DataFrame(yf.download(symbol, start=datetime.datetime.now()-datetime.timedelta(days = 365), interval="1h"))
data['Date'] = data.index
data.reset_index(drop=True,inplace=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
data.head(5)

,Open,High,Low,Close,Adj Close,Volume,Date
0,0.035851,0.035905,0.035729,0.035733,0.035733,0,2021-04-12 00:00:00+00:00
1,0.035739,0.035880,0.035735,0.035856,0.035856,0,2021-04-12 01:00:00+00:00
2,0.035857,0.035857,0.035744,0.035777,0.035777,0,2021-04-12 02:00:00+00:00
3,0.035778,0.035826,0.035634,0.035643,0.035643,0,2021-04-12 03:00:00+00:00
4,0.035619,0.035819,0.035610,0.035791,0.035791,879,2021-04-12 04:00:00+00:00


In [37]:
def grid(df,grids,pct_range,wtime):
    Trades = 0
    Wins = 0
    Max = 0
    Min = 100
    Total = 0
    for i in range(0,len(df)-wtime,24):
        
        buy_orders = []
        sell_orders = []
        entry_price=df['Close'].iloc[i]
        top_range = entry_price*(1+pct_range)
        bottom_range = entry_price*(1-pct_range)
        delta = (top_range - bottom_range)/grids
        Token2=TOKEN
        Token1=Token2/entry_price
        T1_amount = Token1/grids
        for j in range(0,grids+1):
            if bottom_range+(delta*j)<entry_price:
                buy_orders.append(bottom_range+(delta*j))
            elif bottom_range+(delta*j)<=top_range and bottom_range+(delta*j)>entry_price:
                sell_orders.append(bottom_range+(delta*j))
        buy_orders.reverse()
        for n in range(i+1,i+1+wtime):
            actual_price=df['Close'].iloc[n]
            while buy_orders and actual_price<=buy_orders[0]:
                Token1 = Token1 + T1_amount
                Token2 = Token2 - (T1_amount*buy[0]*0.999)
                sell_orders.insert(0,entry_price)
                entry_price = buy_orders[0]
                buy_orders.pop(0)
            while sell_orders and actual_price>=sell_orders[0]:
                Token1 = Token1 - T1_amount
                Token2 = Token2 + (T1_amount*sell_orders[0]*0.999)
                buy_orders.insert(0,entry_price)
                entry_price = sell_orders[0]
                sell_orders.pop(0)
        Result = Token2 + Token1 * actual_price
        Trades = Trades+1
        if Result>TOKEN*2:
            Wins = Wins+1
        if Result>Max:
            Max = Result
        if Result<Min:
            Min = Result
        Total = Total + Result
    return Trades,Wins,Max,Min,Total
            

In [38]:
trades,wins,maximo,minimo,total = grid(data,4,0.1,50*24)
print(trades)
print(wins)
print(maximo)
print(minimo)
print(total/trades)

307
304
0.009757925384151718
0.0038646196474072404
0.004865445157992578


In [39]:
days = [1,2,5,10,15,30,45]
grids = [10,20,30,40,50,60,70,80]
ranges = [5,10,15,20]
f = open("myfile.csv", "a")
f.write('Day,Grid,Range,Trades,Wins,Max,Min,Total\n')
for day in days:
    print(day)
    for gride in grids:
        print(gride)
        for rango in ranges:
            trades,wins,maximo,minimo,total = grid(data,gride,rango/100,day*24)
            f.write(str(day)+','+str(gride)+','+str(rango)+','+str(trades)+','+str(wins)+','+str(maximo)+','+str(minimo)+','+str(total)+'\n')
f.close()


1
10
20
30
40
50
60
70
80
2
10
20
30
40
50
60
70
80
5
10
20
30
40
50
60
70
80
10
10
20
30
40
50
60
70
80
15
10
20
30
40
50
60
70
80
30
10
20
30
40
50
60
70
80
45
10
20
30
40
50
60
70
80
